In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
results_dir_priors = "results/priors/single_layer/tanh/friedman"
results_dir_posteriors = "results/regression/single_layer/tanh/friedman"
#results_dir_posteriors_dst = "results/regression/single_layer/tanh/friedman/full_regularization"

prior_names_and_configs = {"Dirichlet Horseshoe": "dir_hs", "Dirichlet Student T": "dir_stud_t"}
posterior_names = ["Dirichlet Horseshoe tanh", "Regularized Horseshoe tanh", "Dirichlet Student T tanh"]
#posterior_names_and_configs_dst = ["Dirichlet Student T tanh"]
   
prior_fits = {}
posterior_fits = {}

for key, value in prior_names_and_configs.items():
    prior_fit = get_model_fits(
        config=value,
        results_dir=results_dir_priors,
        models=key,
        include_prior=False,
    )
    prior_fits[key] = prior_fit
    
#for key, value in posterior_names_and_configs.items():
posterior_N100_fits = get_model_fits(
    config="Friedman_N100_p10_sigma1.00_seed1",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)

posterior_N200_fits = get_model_fits(
    config="Friedman_N200_p10_sigma1.00_seed2",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)

posterior_N500_fits = get_model_fits(
    config="Friedman_N500_p10_sigma1.00_seed11",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)




In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpmath import hyper
from mpmath import gamma
from scipy.special import poch


p=10
alpha=1.0
b=(p-1)*alpha


def p_kappa_dirichlet_horseshoe(kappa, a_j=1.0, p=10):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    prefactor = (1/np.pi) * (a_j / ((1-kappa) * np.sqrt(kappa) * np.sqrt(1-kappa))) * (1/p)
    c = (-kappa / (1-kappa))*(a_j**2)
    # {}_3F_2([1, 1.1/2, 2.1/1], [1, 3/2], z)
    hyper_val = hyper([p*alpha, (alpha+1)/2, alpha/2 + 1], [(p*alpha+1)/2, p*alpha/2 + 1], c)
    return float(prefactor * hyper_val)

def p_kappa_dirichlet_student_t(kappa, nu=3.0, a_j=1.0, p=10):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    C = gamma((nu+1)/2) * 1/(np.sqrt(np.pi * nu) * gamma(nu/2))
    prefactor = ((a_j**nu) * nu**((nu+1)/2) * kappa**(nu/2 - 1)) / ((1-kappa)**(nu/2 + 1))  * poch(alpha, nu) / poch(p*alpha, nu)
    c = (-kappa / (1-kappa))*(a_j**2)
    # {}_3F_2([1, 1.1/2, 2.1/1], [1, 3/2], z)
    hyper_val = hyper([(nu+1)/2, (alpha+nu)/2, (alpha+nu+1)/2], [(p*alpha+nu)/2, (p*alpha+nu+1)/2], c)
    return float(C * prefactor * hyper_val)

def p_kappa_horseshoe(kappa, a_j=1.0):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    prefactor = (1/np.pi) * (a_j / ((a_j**2-1)*kappa + 1)) * 1/(np.sqrt(kappa) * np.sqrt(1-kappa))
    return float(prefactor)


In [9]:
# Lag kappa-grid
kappa_vals = np.linspace(0.001, 0.999, 500)
p_vals_dirichlet_horseshoe = [p_kappa_dirichlet_horseshoe(k) for k in kappa_vals]
p_vals_dirichlet_horseshoe_medium_low = [p_kappa_dirichlet_horseshoe(k, a_j=0.7) for k in kappa_vals]
p_vals_dirichlet_horseshoe_medium_high = [p_kappa_dirichlet_horseshoe(k, a_j=2) for k in kappa_vals]
p_vals_dirichlet_horseshoe_low = [p_kappa_dirichlet_horseshoe(k, a_j=0.1) for k in kappa_vals]
p_vals_dirichlet_horseshoe_high = [p_kappa_dirichlet_horseshoe(k, a_j=5) for k in kappa_vals]

p_vals_dirichlet_student_t = [p_kappa_dirichlet_student_t(k) for k in kappa_vals]
p_vals_dirichlet_student_t_medium_low = [p_kappa_dirichlet_student_t(k, a_j=0.7) for k in kappa_vals]
p_vals_dirichlet_student_t_medium_high = [p_kappa_dirichlet_student_t(k, a_j=2) for k in kappa_vals]
p_vals_dirichlet_student_t_low = [p_kappa_dirichlet_student_t(k, a_j=0.1) for k in kappa_vals]
p_vals_dirichlet_student_t_high = [p_kappa_dirichlet_student_t(k, a_j=5) for k in kappa_vals]

p_vals_horseshoe = [p_kappa_horseshoe(k) for k in kappa_vals]
p_vals_horseshoe_medium_low = [p_kappa_horseshoe(k, a_j=0.7) for k in kappa_vals]
p_vals_horseshoe_medium_high = [p_kappa_horseshoe(k, a_j=2) for k in kappa_vals]
p_vals_horseshoe_low = [p_kappa_horseshoe(k, a_j=0.1) for k in kappa_vals]
p_vals_horseshoe_high = [p_kappa_horseshoe(k, a_j=5) for k in kappa_vals]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6), sharex=True, sharey=False)

axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_low, label=r"DHS $a_j = 0.1$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_medium_low, label=r"DHS $a_j = 0.7$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe, label=r"DHS $a_j = 1$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_medium_high, label=r"DHS $a_j = 2$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_high, label=r"DHS $a_j = 5$")
axes[0].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[0].set_xlabel(r"$\kappa$")
axes[0].set_title("Dirichlet–Horseshoe")
axes[0].set_ylim((0, 2))
axes[0].legend()

axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_low, label=r"DST $a_j = 0.1$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_medium_low, label=r"DST $a_j = 0.7$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t, label=r"DST $a_j = 1$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_medium_high, label=r"DST $a_j = 2$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_high, label=r"DST $a_j = 5$")
#axes[1].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[1].set_xlabel(r"$\kappa$")
axes[1].set_title("Dirichlet–Student T")
axes[1].set_ylim((0, 7))
axes[1].legend()

axes[2].plot(kappa_vals, p_vals_horseshoe_low, label=r"HS $a_j = 0.1$")
axes[2].plot(kappa_vals, p_vals_horseshoe_medium_low, label=r"HS $a_j = 0.7$")
axes[2].plot(kappa_vals, p_vals_horseshoe, label=r"HS $a_j = 1$")
axes[2].plot(kappa_vals, p_vals_horseshoe_medium_high, label=r"HS $a_j = 2$")
axes[2].plot(kappa_vals, p_vals_horseshoe_high, label=r"HS $a_j = 5$")
#axes[2].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[2].set_xlabel(r"$\kappa$")
axes[2].set_title("Horseshoe")
axes[2].set_ylim((0, 4))
axes[2].legend()

plt.tight_layout()
plt.show()


In [3]:
path = "datasets/friedman/Friedman_N100_p10_sigma1.00_seed1.npz"
data = np.load(path)

In [4]:
import numpy as np

# ---------- 1) Simuler X ----------
def simulate_X(n, P, seed=42):
    rng = np.random.default_rng(seed)
    return rng.uniform(0.0, 1.0, size=(n, P))

# ---------- 2) Aktivering og derivert ----------
def get_activation(activation="tanh"):
    if activation == "tanh":
        phi = np.tanh
        def dphi(a): return 1.0 - np.tanh(a)**2
    elif activation == "relu":
        def phi(a): return np.maximum(0.0, a)
        def dphi(a): return (a > 0.0).astype(a.dtype)
    else:
        raise ValueError(f"Unsupported activation: {activation}")
    return phi, dphi

# ---------- 3) Hovedfunksjon: q for alle trekk ----------
def compute_q_for_fit(cmdstan_mcmc, N=1000, activation="tanh", seed=1, output_index=0, X=None):
    """
    Beregn q_{ell, j} for første-lagsvektene for hver trekk (draw).
    Returnerer:
      q_draws:  (n_draws, H, P)
      q_mean:   (H, P)  – gjennomsnitt over trekk
      X:        (N, P)  – datasettet brukt i beregningen
    """
    # Hent ut variabler fra Stan
    W1_all = cmdstan_mcmc.stan_variable("W_1")            # (draws, P, H)
    WL_all = cmdstan_mcmc.stan_variable("W_L")             # (draws, H, O)
    hb_all = cmdstan_mcmc.stan_variable("hidden_bias")     # (draws, L, H)
    sigma_all = cmdstan_mcmc.stan_variable("sigma")        # (draws,)
    Wint_all = cmdstan_mcmc.stan_variable("W_internal")    # (draws, max(L-1,1), H, H)

    draws, P, H = W1_all.shape
    O = WL_all.shape[2]
    L = hb_all.shape[1]

    if O == 0:
        raise ValueError("W_L has zero output nodes. Expected at least 1.")
    if output_index < 0 or output_index >= O:
        raise ValueError(f"output_index {output_index} out of range 0..{O-1}")

    if X is None:
        X = simulate_X(N, P, seed=seed)

    X_sq = X**2
    phi, dphi = get_activation(activation)

    q_draws = np.empty((draws, H, P), dtype=float)

    for s in range(draws):
        W1 = W1_all[s]            # (P, H)
        WL = WL_all[s]            # (H, O)
        hb = hb_all[s]            # (L, H)
        Wints = Wint_all[s]       # (max(L-1,1), H, H)
        sigma = float(sigma_all[s])

        # ----- Forward pass -----
        a_list = []
        h_list = []

        a = X @ W1 + hb[0]        # (N, H)
        h = phi(a)
        a_list.append(a); h_list.append(h)

        for l in range(1, L):
            Wl = Wints[l-1]       # (H, H)
            a = h @ Wl + hb[l]    # (N, H)
            h = phi(a)
            a_list.append(a); h_list.append(h)

        # ----- Backward: delta_L = d f / d a^(L) -----
        # lineær utgang: df/dh^(L) = WL[:, output_index]
        v = WL[:, output_index]           # (H,)
        delta = dphi(a_list[-1]) * v      # (N, H), broadcast over N

        # Bakover gjennom skjulte lag
        for l in range(L-2, -1, -1):
            Wnext = Wints[l]              # (H, H) – brukes bare hvis L>1
            delta = (delta @ Wnext.T) * dphi(a_list[l]) if L > 1 else delta

        delta1 = delta  # (N, H) == ∂f/∂a^(1)

        # ----- q: (1/sigma^2) * sum_i (delta1[i,ell]^2 * X[i,j]^2) -----
        D_sq = delta1**2                  # (N, H)
        Q = (X_sq.T @ D_sq) / (sigma**2)  # (P, H)
        q_draws[s] = Q.T                  # (H, P)

    q_mean = q_draws.mean(axis=0)         # (H, P)
    return q_draws, q_mean, X

# ---------- 4) Eksempelbruk ----------
# Velg riktig fit-objekt (CmdStanMCMC) fra dict-en din:
prior_q_dhs, prior_q_mean_dhs, X_train = compute_q_for_fit(
    prior_fits['Dirichlet Horseshoe']['Dirichlet Horseshoe']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_dhs, posterior_q_mean_dhs, X_train = compute_q_for_fit(
    posterior_N100_fits['Dirichlet Horseshoe tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

prior_q_dst, prior_q_mean_dst, X_train = compute_q_for_fit(
    prior_fits['Dirichlet Student T']['Dirichlet Student T']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_dst, posterior_q_mean_dst, X_train = compute_q_for_fit(
    posterior_N100_fits['Dirichlet Student T tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_rhs, posterior_q_mean_rhs, X_train = compute_q_for_fit(
    posterior_N100_fits['Regularized Horseshoe tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# ---- choose which node/input to inspect ----
node_idx = 1
input_idx = 1

## Dirichlet Horseshoe

In [6]:
# -------- PRIOR --------
prior_fit = prior_fits['Dirichlet Horseshoe']['Dirichlet Horseshoe']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]  # (draws_prior,)
phi_prior = prior_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]           # (draws_prior,)
q_prior   = prior_q_dhs[:, node_idx, input_idx]                               # (draws_prior,)

# -------- POSTERIOR --------
post_fit = posterior_N100_fits['Dirichlet Horseshoe tanh']['posterior']
tau_post = post_fit.stan_variable("tau")                                      # (draws_post,)
lam_post = post_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]    # (draws_post,)
phi_post = post_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]             # (draws_post,)
q_post   = posterior_q_dhs[:, node_idx, input_idx]                           # (draws_post,)

kappa_DHS_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2) * (phi_prior**2))
kappa_DHS_post = 1.0 / (1.0 + q_post * (tau_post**2) * (lam_post**2) * (phi_post**2))

## Dirichlet Student T

In [7]:
# -------- PRIOR --------
prior_fit = prior_fits['Dirichlet Student T']['Dirichlet Student T']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda")[:, :, node_idx][:, input_idx]  # (draws_prior,)
phi_prior = prior_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]           # (draws_prior,)
q_prior   = prior_q_dhs[:, node_idx, input_idx]                               # (draws_prior,)

# -------- POSTERIOR --------
post_fit = posterior_N100_fits['Dirichlet Student T tanh']['posterior']
tau_post = post_fit.stan_variable("tau")                                      # (draws_post,)
lam_post = post_fit.stan_variable("lambda")[:, :, node_idx][:, input_idx]    # (draws_post,)
phi_post = post_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]             # (draws_post,)
q_post   = posterior_q_dhs[:, node_idx, input_idx]                           # (draws_post,)


kappa_DST_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2) * (phi_prior**2))
kappa_DST_post = 1.0 / (1.0 + q_post * (tau_post**2) * (lam_post**2) * (phi_post**2))

## Regularized Horseshoe

In [8]:
prior_fit = prior_fits['Dirichlet Horseshoe']['Dirichlet Horseshoe']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]  # (draws_prior,)
q_prior   = prior_q_dhs[:, node_idx, input_idx]                               # (draws_prior,)

post_rhs_fit = posterior_N100_fits['Regularized Horseshoe tanh']['posterior']
tau_rhs_post = post_rhs_fit.stan_variable("tau")                                      # (draws_post,)
lam_rhs_post = post_rhs_fit.stan_variable("lambda_tilde")[:, :, node_idx][:, input_idx]    # (draws_post,)
q_rhs_post   = posterior_q_rhs[:, node_idx, input_idx]                           # (draws_post,)

kappa_HS_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2))
kappa_HS_post = 1.0 / (1.0 + q_rhs_post * (tau_rhs_post**2) * (lam_rhs_post**2))

In [ ]:
# -------- Plot: prior vs posterior overlays --------
bins = np.linspace(0.0, 1.0, 40)  # common bins so densities are comparable

fig, axes = plt.subplots(1, 3, figsize=(8, 4), sharex=True, sharey=True)

# Dirichlet–Horseshoe
axes[0].hist(kappa_DHS_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[0].hist(kappa_DHS_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[0].set_title("Dirichlet Horseshoe")
axes[0].set_xlabel(r"$\kappa_{\ell j}$")
axes[0].set_ylabel("Density")
axes[0].legend()

# Dirichlet–Horseshoe
axes[1].hist(kappa_DST_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[1].hist(kappa_DST_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[1].set_title("Dirichlet Student T")
axes[1].set_xlabel(r"$\kappa_{\ell j}$")
axes[2].legend()

# Horseshoe
axes[2].hist(kappa_HS_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[2].hist(kappa_HS_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[2].set_title("Horseshoe")
axes[2].set_xlabel(r"$\kappa_{\ell j}$")
axes[2].legend()

plt.suptitle(f"Node {node_idx}, input {input_idx}: Prior vs Posterior")
plt.tight_layout()
plt.show()

In [ ]:
# -------- Plot: prior vs posterior overlays --------
bins = np.linspace(0.0, 1.0, 60)  # common bins so densities are comparable

fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

# Dirichlet–Horseshoe
axes[0].hist(kappa_DST_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[0].hist(kappa_DST_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[0].set_title("Dirichlet Student T")
axes[0].set_xlabel(r"$\kappa_{\ell j}$")
axes[0].set_ylabel("Density")
axes[0].legend()

# Horseshoe
axes[1].hist(kappa_HS_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[1].hist(kappa_HS_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[1].set_title("Horseshoe")
axes[1].set_xlabel(r"$\kappa_{\ell j}$")
axes[1].legend()

plt.suptitle(f"Node {node_idx}, input {input_idx}: Prior vs Posterior")
plt.tight_layout()
plt.show()